### Retriever

In [4]:
# ! pip install bs4

In [6]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain.embeddings import HuggingFaceBgeEmbeddings


model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeddings)

retriever = vectorstore.as_retriever()

### Importing Togther API Mistal Model using the utils

In [8]:
from utils.llm import LLM
llm = LLM().get_llm_together()

### Hypotherical Document/Answer Generation Step

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# HyDE document genration
template = """Please write a concise passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)



generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

' Task decomposition is a method used in large language model (LLM) agents to break down complex tasks into smaller, manageable sub-tasks. This approach allows the agent to process and solve complex problems more efficiently by focusing on one sub-task at a time. By decomposing a task into smaller parts, the agent can also apply its knowledge and skills more effectively, reducing the likelihood of errors and improving overall performance. For example, a task to write a research paper could be decomposed into sub-tasks such as researching the topic, outlining the paper, writing the introduction, and so on. Each sub-task can then be assigned to a specific module or component of the LLM agent, allowing for a more efficient and effective solution to the overall task.'

### Hypothetical Answer + Question for Retrieval ---> Context

In [10]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via 

### Pass the Question + Retrieved Documents for better RAG

In [11]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

output = final_rag_chain.invoke({"context":retireved_docs,"question":question})

print(output)


Answer: Task decomposition is a process in LLM agent systems where complex tasks are broken down into smaller, manageable tasks. This can be done using simple LLM prompts, task-specific instructions, or human inputs. The goal is to make the tasks more manageable for the LLM and provide insights into the model's thinking process. Techniques like Chain of Thought and Tree of Thoughts can be used to explore multiple reasoning possibilities at each step. Task decomposition is an essential part of LLM-powered autonomous agent systems as it helps the agent to plan and execute tasks more effectively.
